In [1]:
from hotels50k import Hotels50kDataset, UseOriginalTestSplitManager, compress_serialize
from powerful_benchmarker.split_managers import IndexSplitManager
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
dataset = Hotels50kDataset(root='/data/thesis/Hotels-50K', target='hotels', download=True)

Loading image folders
Concatting dataset
Getting labels
Done loading dataset


In [5]:
len(dataset)

867

In [6]:
len(dataset.original_train_dataset), len(dataset.original_test_dataset)

(859, 8)

In [7]:
dataset[858]

{'data': <PIL.Image.Image image mode=RGB size=640x426 at 0x7F5BE54ABC10>,
 'label': 49623}

In [8]:
dataset.labels.shape

(867,)

In [9]:
len(dataset.original_train_dataset), len(dataset.original_test_dataset)

(859, 8)

In [10]:
dataset.original_train_dataset.keys[-10:]

[b'849',
 b'850',
 b'851',
 b'852',
 b'853',
 b'854',
 b'855',
 b'856',
 b'857',
 b'858']

In [11]:
len(dataset.train_chains)

859

In [12]:
import numpy as np

In [13]:
np.where(dataset.labels==1)

(array([], dtype=int64),)

In [14]:
assert list(dataset.labels[0:100]) == [dataset[i][1] for i in range(0, 100)]

KeyError: 1

In [15]:
ds = dataset

# first level of dictionary is transform type
# second level consists of separate dataset objects for each split
datasets = {"eval": {"train": ds,
                    "val": ds,
                    "test": ds}
            }

helper_split_manager = UseOriginalTestSplitManager()
split_manager = IndexSplitManager(num_training_partitions=4, num_training_sets=4, helper_split_manager=helper_split_manager)

split_manager.create_split_schemes(datasets)

test_set = split_manager.get_dataset("eval", "test")

for name in split_manager.split_scheme_names:
    split_manager.set_curr_split_scheme(name)
    print("CURR SPLIT SCHEME =", split_manager.curr_split_scheme_name)
    print("train len", len(split_manager.get_dataset("eval", "train")))
    print("val len", len(split_manager.get_dataset("eval", "val")))
    print("test len", len(split_manager.get_dataset("eval", "test")))

    print("train labels", split_manager.get_label_set("eval", "train"))
    print("val labels", split_manager.get_label_set("eval", "val"))
    print("test labels", split_manager.get_label_set("eval", "test"))
    
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "val").indices) == set()
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()
    assert set(split_manager.get_dataset("eval", "val").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()


CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_0
train len 644
val len 215
test len 8
train labels {72929, 37176, 43873, 20613, 6053, 33809, 52435, 43156, 33973, 25492, 49623, 43512, 31836, 5343}
val labels {47111, 46313, 100657, 44852, 31836}
test labels {37176, 100657}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_1
train len 644
val len 215
test len 8
train labels {72929, 43873, 49623, 6053, 47111, 46313, 100657, 33809, 52435, 44852, 43156, 33973, 25492, 37176, 31836, 5343}
val labels {43512, 43156, 31836, 20613}
test labels {37176, 100657}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_2
train len 644
val len 215
test len 8
train labels {43873, 20613, 47111, 46313, 100657, 33809, 52435, 44852, 43156, 25492, 49623, 43512, 31836}
val labels {72929, 6053, 52435, 43156, 33973, 37176, 5343}
test labels {37176, 100657}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_3
train len 645
val len 214
test len 8
train labels {72929, 37176, 20613, 6053, 47111, 46313, 100657, 52435, 44

In [16]:
train_ds = split_manager.get_dataset("eval", "train")
batch_size = 32

def collate(stuff):
    return [s['data'] for s in stuff]

loader = DataLoader(train_ds, shuffle=True, batch_size=batch_size,  num_workers=4, collate_fn=collate)

for batch in tqdm(loader):
    assert len(batch) > 0

100%|██████████| 21/21 [00:04<00:00,  4.50it/s]
